# Pravděpodobnostní klasifikace (Bayesův klasifikátor)

Úloha zaměřená na implementaci základních pravděpodobnostních metod pro klasifikaci.


## Data
Rozdělení dat uvnitř tříd odpovídá normálnímu rozdělení


In [3]:
import usu
import numpy as np

npzfile = np.load('data/data_11.npz')
npzfile.files


['testData', 'testRef', 'trainData', 'trainRef']

In [4]:
testData = npzfile['testData']
testRef = npzfile['testRef']

trainData = npzfile['trainData']
trainRef = npzfile['trainRef']

nClasses = len(np.unique(trainRef))

trainData.shape,trainRef.shape, testData.shape, testRef.shape


((1900, 2), (1900, 1), (100, 2), (100, 1))

### Výpočet úspěšnosti
$$ accuracy = \frac{\text{počet správně klasifikovaných objektů}}{\text{počet všech klasifikovaných  objektů}} $$

Apriorní pravděpodobnost třídy:

$$ P_{ap}(class) = \frac{\text{počet prvků třídy}}{\text{počet všech prvků všech tříd}} $$



Hustota pravděpodobnosti Gaussova rozdělení pro jednorozměrný příznakový vektor:

$$ f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp{-\frac{(x-\mu)^2}{2 \sigma^2}} $$

kde $\mu$ je střední hodnota a $\sigma^2$ je rozptyl

Po logoritmizaci dostaneme:
$$ L(x) = -\frac{1}{2} \log(2 \pi \sigma^2) -\frac{(x-\mu)^2}{2 \sigma^2} $$

Pro soubor N vzorků dat:

$$ L(X) = \sum_{i=0}^{N}{ -\frac{1}{2} \log(2 \pi \sigma^2)} -\frac{1}{2}\sum_{i=0}^{N}{\frac{(x_{i}-\mu)^2}{\sigma^2} } $$



Celkově:

$$ L(class, X) = -\frac{1}{2} \sum_{i=0}^{N}{  \log(2 \pi \sigma_{class}^2)} -\sum_{i=0}^{N}{\frac{(x_{i}-\mu_{class})^2}{2 \sigma_{class}^2} } + \log P_{ap}(class)$$

kde $ Z = -\frac{1}{2} \sum_{i=0}^{N}{ \log(2 \pi \sigma_{class}^2)} + \log P(class) = -\frac{N}{2} \log(2 \pi \sigma_{class}^2) + \log P(class) $ je pak pro každou třídu konstanta a je možné si je předpočítat. 



Výpočet pro vícerozměrný příznakový vektor a plnou i diagonální kovariační matici viz. přednášky ....

In [165]:
fixeds = []
means = []
determinants = []
covariances = []

for c in range(nClasses):
    # Pro vypočet průmeru, variance a kovariační matice pro každou třídu použijte funkce z numpy.

    # Pro každou třídy si předpočítejte všechny potřebné konstanty.

    # Věrohodnost počítejte v logaritmické míře

    print("TŘÍDA", c)
    mean = np.mean(trainData[trainRef.flatten() == c], axis=0)
    cov = np.cov(trainData[trainRef.flatten() == c].T)
    ap = len(trainData[trainRef.flatten() == c]) / len(trainData)
    # print("Průměr")
    # print(mean)
    # print("Kovarianční matice")
    # print(cov)
    # print("Apriorní pravděpodobnost")
    # print(ap)
    # print("------------")

    # cov = np.diag(np.diag(cov))

    means.append(mean)
    covariances.append(cov)
    det = np.linalg.det(cov)
    determinants.append(det)
    fixed = -1/2*np.log(2*np.pi*det) + np.log(ap)
    fixeds.append(fixed)

predRef = []

for i in range(len(testData)):
    preds = []
    for c in range(nClasses):
        p = -1/2 * (testData[i] - means[c]).T @ np.linalg.inv(covariances[c]) @ (testData[i] - means[c]) + fixeds[c]
        preds.append(p)
    print(preds)
    print(testRef[i], np.argmax(preds))
    predRef.append(np.argmax(preds))
        


TŘÍDA 0
TŘÍDA 1
[-4.765764513315255, -8.924867422294163]
[0.] 0
[-6.118626035419349, -11.17904113168522]
[0.] 0
[-6.377893826250401, -8.009554299053924]
[0.] 0
[-7.0794182965258505, -7.442809261496703]
[0.] 0
[-5.199218433885846, -12.533796275968587]
[0.] 0
[-5.644948274046925, -8.487487426898133]
[0.] 0
[-6.419051539012298, -9.061481540819187]
[0.] 0
[-7.753079830560423, -6.62577162230661]
[0.] 1
[-6.197004731121068, -8.651960707886863]
[0.] 0
[-5.979170705925123, -7.080716767885435]
[0.] 0
[-4.397365021788122, -11.230960640542458]
[0.] 0
[-4.740709153567383, -11.142010603012109]
[0.] 0
[-8.134902379261309, -11.450007013515906]
[0.] 0
[-7.556211068040037, -11.340075432593821]
[0.] 0
[-5.259944702529064, -7.664197638489427]
[0.] 0
[-6.966317986068792, -7.8216277059512604]
[0.] 0
[-4.892845857943933, -8.145736504087493]
[0.] 0
[-4.312674102192417, -9.852227714043565]
[0.] 0
[-6.7062489104598715, -6.713163476060233]
[0.] 0
[-5.598182691015492, -7.350078836704379]
[0.] 0
[-8.1258103512200

In [163]:
def accuracy(testRef, predRef):

    return (predRef==testRef.T).mean() * 100


In [166]:
print(accuracy(testRef, predRef))

96.0
